In [7]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as pl

import sys
sys.path.insert(0,'..')

from starreco.model import *
#from starreco.data import *

%load_ext autoreload
%autoreload 2

In [2]:
user_x = torch.FloatTensor(3760,10).random_(0, 5)
user_train_ds = TensorDataset(user_x, user_x)
user_train_dl = DataLoader(user_train_ds, batch_size = 1024, num_workers = 8)

user_mda = mDA(10, 5)
pl.Trainer(gpus = 1, max_epochs = 500, progress_bar_refresh_rate = 50, logger = False)\
.fit(user_mda, user_train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/kyleong/anaconda3/envs/starreco/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 55    
1 | decoder | Sequential | 60    
---------------------------------------
115       Trainable params
0         Non-trainable params
115       Total params
0.000     Total estimated model params size (MB)


Epoch 499: 100%|██████████| 4/4 [00:00<00:00, 18.62it/s, loss=1.31e+04, train_loss_step=1.52e+4, train_loss_epoch=1.31e+4]


1

In [3]:
item_x = torch.FloatTensor(6040,7).random_(0, 5)
item_train_ds = TensorDataset(item_x, item_x)
item_train_dl = DataLoader(item_train_ds, batch_size = 1024, num_workers = 8)

item_mda = mDA(7, 5)
pl.Trainer(gpus = 1, max_epochs = 500, progress_bar_refresh_rate = 50, logger = False)\
.fit(item_mda, item_train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 40    
1 | decoder | Sequential | 42    
---------------------------------------
82        Trainable params
0         Non-trainable params
82        Total params
0.000     Total estimated model params size (MB)


Epoch 499: 100%|██████████| 6/6 [00:00<00:00, 24.80it/s, loss=3.24e+03, train_loss_step=3.38e+3, train_loss_epoch=3.29e+3]


1

In [4]:
mdacf = mDACF(user_mda, item_mda, 
              weight_decay = 0) # Weight decay > 0 will updates freezed weights

for name, param in user_mda.named_parameters():
    print(name, param)
print("---")
for name, param in item_mda.named_parameters():
    print(name, param)
print("---")
for name, param in mdacf.named_parameters():
    print(name, param)

encoder.0.weight Parameter containing:
tensor([[-2.7126e+00,  1.1661e+00, -1.2689e+00, -2.5834e+00,  3.9964e-01],
        [ 1.2861e+00, -2.0164e+00, -1.6569e+00,  1.3712e+00, -2.5131e+00],
        [-5.6672e-01, -1.6256e+00, -1.7292e+00,  1.2545e+00,  1.4366e+00],
        [-2.5882e+00, -1.5447e+00, -8.6207e-01,  4.4134e-01, -1.1240e+00],
        [ 1.1901e+00,  3.0691e-01,  1.4074e+00,  1.2434e+00,  6.7008e-01],
        [ 9.7806e-02,  1.6562e+00, -1.3372e+00, -7.7548e-01,  6.4279e-01],
        [-2.6087e+00, -1.7731e-03,  2.3472e-01, -9.3453e-01,  1.0998e+00],
        [ 2.7469e-01, -2.7183e+00,  1.7429e+00, -1.9194e+00, -2.5782e+00],
        [ 6.8437e-01,  5.0495e-01, -1.8593e+00, -2.2644e+00, -1.2176e+00],
        [-1.5124e+00,  3.3837e-01,  1.1705e+00, -2.6118e+00,  7.3654e-01]],
       requires_grad=True)
encoder.0.bias Parameter containing:
tensor([0.4306, 0.3943, 0.4182, 0.4022, 0.3303], requires_grad=True)
decoder.0.weight Parameter containing:
tensor([[-2.5179,  1.1604, -0.3327, -2

In [5]:
X = torch.FloatTensor(1000000,17).random_(0, 5)
y = torch.FloatTensor(1000000).random_(1,6)
train_ds = TensorDataset(X,y)
train_dl = DataLoader(train_ds, batch_size = 1024, num_workers = 8)

pl.Trainer(gpus = 1, max_epochs = 100, progress_bar_refresh_rate = 50, logger = False)\
.fit(mdacf, train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name     | Type | Params
----------------------------------
0 | user_mda | mDA  | 115   
1 | item_mda | mDA  | 82    
----------------------------------
95        Trainable params
102       Non-trainable params
197       Total params
0.001     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 977/977 [00:06<00:00, 150.43it/s, loss=2, train_loss_step=2.050, train_loss_epoch=2.000]   


1

In [6]:
for name, param in user_mda.named_parameters():
    print(name, param)
print("---")
for name, param in item_mda.named_parameters():
    print(name, param)
print("---")
for name, param in mdacf.named_parameters():
    print(name, param)

encoder.0.weight Parameter containing:
tensor([[-4.2298e+00,  2.8513e-01, -1.4298e+00,  4.3226e-04,  2.6061e-01],
        [ 7.8817e-01, -1.0308e+00, -2.0594e+00, -1.8189e-03, -2.7300e+00],
        [-1.3828e+00, -1.9681e+00, -1.7320e+00,  4.2058e-03,  4.7564e-01],
        [-2.0084e+00, -9.7755e-01, -8.2517e-01, -4.5225e-05, -1.6357e+00],
        [ 6.5722e-01, -4.5356e-02,  3.7121e-01, -1.0810e-03, -1.5999e-01],
        [ 7.7276e-02,  7.6082e-01, -1.9565e+00, -7.8190e-04, -3.7111e-02],
        [-1.8489e+00, -9.9323e-01, -3.1014e-01,  3.4221e-03, -6.5762e-02],
        [ 1.8050e-01, -2.8665e+00,  8.2338e-01,  7.1886e-04, -2.6109e+00],
        [ 5.4308e-01,  6.5206e-01, -2.5858e+00, -4.2955e-03, -1.1528e+00],
        [-3.5005e+00,  1.3473e-01,  7.6172e-01, -3.1266e-03,  1.5072e-01]],
       requires_grad=True)
encoder.0.bias Parameter containing:
tensor([-0.0705,  0.0133, -0.2966,  4.1258, -0.5688], requires_grad=True)
decoder.0.weight Parameter containing:
tensor([[-2.5179,  1.1604, -0.332